In [21]:
!pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!apt-get install -y -qq glpk-utils

In [23]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

Modeling the problem to a Linear Programming problem:

The size of each side is the constraint for the xi corresponding to that side. The xi for each side, gets a value of 0 or 1, and demonstrates whether or not that side is in the path.

x1 : AB = 2

x2 : AC = 7

x3 : BC = 10

x4 : BE = 30

x5: CB = 10

x6: CD = 8

x7: DE = 5

In [24]:
model = pyo.ConcreteModel()

model.x = pyo.VarList(within=pyo.Binary)
x = model.x
for i in range(7):
  x.add()

Starting from A, it has to choose AB or AC (x1+x2 = 1)

In [25]:
model.c = pyo.ConstraintList()
model.c.add(expr= x[1] + x[2] == 1)

If it chooses AB, it has to choose BC or BE next.

x1 = 1 , x3 + x4 = 1

In [26]:
model.c.add(expr= x[3] + x[4] == x[1])

If it chooses AC, it has to choose CB or CD next.

x2 = 1 , x5 + x6 = 1

In [27]:
model.c.add(expr= x[5] + x[6] == x[2])

If it chooses CD, it will have to choose DE to reach the destination.

In [28]:
model.c.add(expr= x[6] == x[7])

Finally, it has to choose between BE or DE to get to E. (x4+x7 = 1)

In [29]:
model.c.add(expr= x[4] + x[7] == 1)

In [30]:
model.obj = pyo.Objective(expr = 2*x[1] + 7*x[2] + 10*x[3] +
                          30*x[4] + 10*x[5] + 8*x[6] + 5*x[7], sense = minimize)
opt = SolverFactory('glpk')
opt.solve(model)
model.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}

1 Var Declarations
    x : Size=7, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None : 

In [32]:
min_distance = pyo.value(x[1])*2 + pyo.value(x[2])*7 + pyo.value(x[3])*10 + pyo.value(x[4])*30 + pyo.value(x[5])*10 + pyo.value(x[6])*8 + pyo.value(x[7])*5
print('min distance = ',min_distance)

min distance =  20.0
